### **LlamaIndex x Mistral AI x BeyondLLM Evaluation**

In this notebook, we evaluate the performance of LlamaIndex, Mistral AI, and BeyondLLM in a unified retrieval and generation pipeline.

- **Objective:** Integrate LlamaIndex, Mistral AI, and BeyondLLM to assess and improve model performance.
- **Setup:**
  - Load and prepare data using LlamaIndex.
  - Configure the Mistral AI model and retrieval system.
- **Steps:**
  - Set up data processing and indexing with LlamaIndex.
  - Initialize the Mistral AI model for response generation.
  - Configure the retriever to handle queries effectively.
  - Generate and evaluate responses to determine model performance.
- **Outcome:** Gain insights into the effectiveness of combining these technologies for enhanced model evaluation and performance assessment.

## Setup and Installation

In [ ]:
!pip install llama-index-vector-stores-chroma
!pip install llama-index
!pip install llama-index-embeddings-fastembed
!pip install llama-index-llms-huggingface-api
!pip install beyondllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.5 MB/s eta 0:00:00


  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.3 MB/s eta 0:00:00
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━


## Set Up Hugging Face API Token

In [ ]:
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass("API:")

# Set the API token in the environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

API:··········


## Import Libraries
Import the necessary libraries to work with the LlamaIndex framework, vector stores, embeddings, and evaluation metrics:

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
import chromadb
from beyondllm.utils import CONTEXT_RELEVENCE, GROUNDEDNESS, ANSWER_RELEVENCE
import re
import numpy as np
import pysbd

## Load the  Documents
Here we are using a heart health document that can be downloaded [here](https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf).


In [ ]:
documents = SimpleDirectoryReader("/content/sample_data/Data").load_data()

/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


## Configure Embeddings and Language Models


In [ ]:
embed_model = FastEmbedEmbedding(model_name="thenlper/gte-large")
llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mistral-7B-Instruct-v0.2", token=HUGGINGFACEHUB_API_TOKEN
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

# Initialize Chroma Vector Store
Chroma is a vector store used to manage and query vectorized embeddings. Follow these steps to set up and use Chroma with LlamaIndex.

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


#Define Evaluation Functions
Implement functions to evaluate different aspects of the language model's performance:

## Context Relevancy
This function evaluates how relevant the context is to the query. It calculates the average relevancy score across all context segments:

In [ ]:
def get_context_relevancy(llm, query, context):
    total_score = 0
    score_count = 0

    for content in context:
        score_response = llm.complete(CONTEXT_RELEVENCE.format(question=query, context=content))

        score_str = score_response.text
        score = float(extract_number(score_str))
        total_score += score
        score_count += 1

    average_score = total_score / score_count if score_count > 0 else 0
    return f"Context Relevancy Score: {round(average_score, 1)}"



## Answer Relevancy
This function assesses how relevant the generated answer is to the query:

In [ ]:
def get_answer_relevancy(llm, query, response):
    score_response = llm.complete(ANSWER_RELEVENCE.format(question=query, context=response))
    score_str = score_response.text
    return f"Answer Relevancy Score: {score_str}"



## Groundedness
Evaluate how grounded the response is based on the context provided. This function calculates the average groundedness score for each statement in the response:

In [ ]:
def get_groundedness(llm, query, context, response):
    total_score = 0
    score_count = 0

    statements = sent_tokenize(response)

    for statement in statements:
        score_response = llm.complete(GROUNDEDNESS.format(statement=statement, context=" ".join(context)))
        score_str = score_response.text
        score = float(extract_number(score_str))
        total_score += score
        score_count += 1

    average_groundedness = total_score / score_count if score_count > 0 else 0
    return f"Groundedness Score: {round(average_groundedness, 1)}"


## Execute Evaluation
Perform a query using the pipeline and evaluate its performance with the defined functions:

In [ ]:
query = "what doesnt cause heart diseases"

retrieved_documents = query_engine.retrieve(query)
context = [doc.node.text for doc in retrieved_documents]

response = query_engine.query(query)

print(get_context_relevancy(llm, query, context))
print(get_answer_relevancy(llm, query, response.response))
print(get_groundedness(llm, query, context, response.response))

Context Relevancy Score: 3.0
Answer Relevancy Score: 6

The context provides some information about factors that can help protect against heart disease, but it does not provide a definitive list of things that do not cause heart disease. Therefore, the relevance score is 6.
Groundedness Score: 6.0


In [ ]:
query = "what is the capital of turkey"

retrieved_documents = query_engine.retrieve(query)
context = [doc.node.text for doc in retrieved_documents]

response = query_engine.query(query)

print(get_context_relevancy(llm, query, context))
print(get_answer_relevancy(llm, query, response.response))
print(get_groundedness(llm, query, context, response.response))

Context Relevancy Score: 0.0
Answer Relevancy Score: 0
Groundedness Score: 5.0


## Conclusion

The evaluation of the LlamaIndex pipeline with BeyondLLM revealed the following results for two example queries:

### Query 1: "What doesn't cause heart diseases?"

- **Context Relevancy Score:** 3.0 - The context is partially relevant but doesn’t fully address the query.
- **Answer Relevancy Score:** 6 - The response provides some relevant information but lacks a definitive list.
- **Groundedness Score:** 6.0 - The response is reasonably grounded based on the context.

### Query 2: "What is the capital of Turkey?"

- **Context Relevancy Score:** 0.0 - The context is not relevant to the query.
- **Answer Relevancy Score:** 0 - The response does not answer the query.
- **Groundedness Score:** 5.0 - The response shows some level of groundedness, though it doesn't match the query.

These results highlight how context and response relevance affect evaluation scores. Proper alignment with the query improves these scores, while poor relevancy and incorrect answers lead to lower scores.